<div style='text-align:center'>
<h1><b>به نام خدا</b></h1>
<h2>تمرین اول بیگ دیتا</h2>
<h3>وحید ملکی</h3>
</div>

<div dir='rtl'>
<h3>بخش دوم</h3>
</div>

<div dir='rtl'>
<h3>توضیحات:</h3>
ابتدا کتابخانه های لازم را ایمپورت میکنیم.
</div>

In [51]:
import pandas as pd
import pymongo

<div dir='rtl'>
سپس با پایگاه داده ارتباط برقرار میکنیم.
</div>

In [52]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["myDataBase"]
mycollec = mydb['data']

<div dir='rtl'>
در کد زیر داده ها را از فایل data.csv میخوانیم و به دیتابیس اضافه میکنیم.
ورودی تابع insert_many باید به صورت چندین دیکشنری باشد که کلید ها اسم ستون ها باید باشد و value مقدار آن باید باشد به همین دلیل هنگام خواندن فایل آن را ترنسپوز میکنیم تا به شکل مورد نیاز تبدیل شود.
</div>

In [53]:
df = pd.read_csv('./Required datasets/data.csv').T
data = df.to_dict().values()
x = mycollec.insert_many(data)

<div dir='rtl'>
لیستی از وضعیت کلی برای روز چهارم هفته باشگاه به ترتیب تعداد افراد
</div>

In [54]:
pipeline = [{'$match':{'day_of_week':4}}, 
            {'$sort':{'number_people':-1}}]

for i in mycollec.aggregate(pipeline):
    print(i)

{'_id': ObjectId('67210c9598d753d054f316d9'), 'number_people': 124, 'date': '2015-09-11 17:59:03-07:00', 'timestamp': 64743, 'day_of_week': 4, 'is_weekend': 0, 'is_holiday': 0, 'temperature': 70.13, 'is_start_of_semester': 0, 'is_during_semester': 1, 'month': 9, 'hour': 17}
{'_id': ObjectId('67210c9598d753d054f31448'), 'number_people': 118, 'date': '2015-09-04 18:01:01-07:00', 'timestamp': 64861, 'day_of_week': 4, 'is_weekend': 0, 'is_holiday': 0, 'temperature': 66.25, 'is_start_of_semester': 1, 'is_during_semester': 1, 'month': 9, 'hour': 18}
{'_id': ObjectId('67210c9598d753d054f31441'), 'number_people': 117, 'date': '2015-09-04 16:40:13-07:00', 'timestamp': 60013, 'day_of_week': 4, 'is_weekend': 0, 'is_holiday': 0, 'temperature': 68.05, 'is_start_of_semester': 1, 'is_during_semester': 1, 'month': 9, 'hour': 16}
{'_id': ObjectId('67210c9598d753d054f33683'), 'number_people': 116, 'date': '2015-12-04 17:00:13-08:00', 'timestamp': 61213, 'day_of_week': 4, 'is_weekend': 0, 'is_holiday': 0

<div dir='rtl'>
مجموع افرادی که در ماه هشتم به باشگاه می ایند.
</div>

In [55]:
pipeline = [{'$group':{'_id':'$month', 'sum_people': {'$sum':'$number_people'}}},
            {'$match':{'_id':8}}]

list(mycollec.aggregate(pipeline))

[{'_id': 8, 'sum_people': 162845}]

<div dir='rtl'>
مجموع افرادی که در تعطیلات به باشگاه می ایند.
</div>

In [56]:
pipeline = [
    {'$match':{'is_holiday':1}},
    {'$group':{'_id':None, 'sum_people':{'$sum':'$number_people'}}}
]

list(mycollec.aggregate(pipeline))

[{'_id': None, 'sum_people': 1203}]

<div dir='rtl'>
پنج روز از شلوغ ترین روز های باشگاه به ترتیب

در این بخش تاریخ که به صورت رشته است را از طریق datefromstring به تاریخ تبدیل میکنیم سپس هنگام تبدیل دوباره به رشته با استفاده از datetostring فقط سال و ماه و روز را انتخاب میکنیم و گروهبندی را با آن انجام میدهیم در اینصورت هر روز یک گروه میشود و میتوانیم به تعداد افراد در هر روز خاص دسترسی داشته باشیم.
</div>

In [57]:
pipeline = [
    {'$group': {'_id': {'$dateToString': {'format': "%Y-%m-%d", 'date': {"$dateFromString": {"dateString": "$date"}}}}, 
                'sum_people': {'$sum':'$number_people'}}},
    {'$sort': {'sum_people':-1}},
    {'$limit': 5},
]
for i in list(mycollec.aggregate(pipeline)):
    print(i)

{'_id': '2016-01-21', 'sum_people': 6978}
{'_id': '2016-01-20', 'sum_people': 6848}
{'_id': '2015-08-26', 'sum_people': 6559}
{'_id': '2016-01-29', 'sum_people': 6239}
{'_id': '2015-09-24', 'sum_people': 6229}


<div dir='rtl'>
میانگین دما و وضعیت تعطیلی در روز های خلوت باشگاه به ترتیب تاریخ

از همان ایده کد قبل استفاده میکنیم ولی این بار مرتب سازی را برعکس در نظر میگیریم و همچنین ستون های درخواست شده را نیز به خروجی اضافه مبکنبم.
</div>

In [58]:
pipeline = [
    {'$group': {'_id': {'$dateToString': {'format': "%Y-%m-%d", 'date': {"$dateFromString": {"dateString": "$date"}}}}, 
                'is_holiday':{'$avg':'$is_holiday'},
                'sum_people': {'$sum':'$number_people'},
                'avg_temp':{'$avg':'$temperature'},}},
    {'$sort': {'sum_people':1}},
    {'$limit': 5},
    {'$sort': {'_id':1}},
]
for i in list(mycollec.aggregate(pipeline)):
    print(i)

{'_id': '2015-12-25', 'is_holiday': 0.0, 'sum_people': 14, 'avg_temp': 47.3546}
{'_id': '2016-12-25', 'is_holiday': 0.0, 'sum_people': 33, 'avg_temp': 47.62062068965517}
{'_id': '2017-01-06', 'is_holiday': 0.0, 'sum_people': 6, 'avg_temp': 41.47}
{'_id': '2017-01-20', 'is_holiday': 0.0, 'sum_people': 42, 'avg_temp': 51.22}
{'_id': '2017-02-14', 'is_holiday': 0.0, 'sum_people': 42, 'avg_temp': 54.7}


<div dir='rtl'>
مجموعه افرادیکه در روز دوم و ماه دهم از ساعت 17 تا 22 به باشگاه میایند.

در این بخش نیز فیلتر های ماه و ساعت را به راحتی اضافه میکنیم فقط در مشخص کردن روز از زیر زشته استفاده میکنیم.
</div>

In [59]:
pipeline = [
    {'$match':{'month':10, 
               'hour':{'$gte':17, '$lte':22}, 
               '$expr':{'$eq':[{'$substr':['$date', 8, 2]}, '02']}}},
    {'$group':{'_id':None, 'sum_people':{'$sum':'$number_people'}}}
]

for i in list(mycollec.aggregate(pipeline)):
    print(i)

{'_id': None, 'sum_people': 3465}
